In [ ]:
import csv
import requests
import json
import random
from random import shuffle
import pandas as pd
import progressbar
import os.path

import pyvips
import numpy as np
import  requests, json, os
import glob
import io
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import gc
import random

import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
from PIL import Image

path = "/home/amir/project/tcga_patches/"
converted_files = "/home/amir/project/pytorch-CycleGAN-and-pix2pix/tmad/all_stain_cyclegan/test_1/images/"

X=[]
X_processed=[]
Y=[]
X_train = None
Y_train = None

classes = {}

im = ""

def adjust_image(im):
    imgs = im.resize((299, 299), Image.ANTIALIAS)
    return imgs

for directory in os.listdir(path):
    print(directory)
    for file in os.listdir(path + directory)[:300]:
        converted_file = converted_files + file[:-4] + "_fake_B.png"        
        if(os.path.exists(converted_file)):
            img = Image.open(path + directory + "/" + file).convert("RGB")
            img = adjust_image(img)
            features=np.array(img, dtype=np.float32)/255.0
            processed_img = Image.open(converted_file)
            processed_img = adjust_image(processed_img)
            processed_feature=np.array(processed_img, dtype=np.float32)/255.0
            X_processed.append(processed_feature)
            X.append(features)
            if directory in classes:
                class_id = classes[directory]
            else:
                class_id = len(classes.keys())
                classes[directory] = class_id
            Y.append(class_id)


print("Read all of data.")
            
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
X_processed_train, X_processed_test, y_processed_train, y_processed_test = train_test_split(X_processed, Y, test_size=0.33, random_state=42)

X_processed_train = np.array(X_processed_train)
X_processed_test = np.array(X_processed_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_processed_train, X_processed_test, y_processed_train, y_processed_test = train_test_split(X_processed, Y, test_size=0.33, random_state=42)
X_processed_train = np.array(X_processed_train)
X_processed_test = np.array(X_processed_test)
X_processed = ""

In [ ]:
X_train.shape
np.bincount(y_test)

In [ ]:
plt.imshow(X_test[11])

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
NUM_CLASSES = 5

def inceptionv3_model_fn(features, labels, mode):
    # Load Inception-v3 model.
    module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1")
    input_layer = features["x"]
    outputs = module(input_layer)
    logits = tf.layers.dense(inputs=outputs, units=NUM_CLASSES)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    accuracy = tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])
    tf.summary.scalar("accuracy", accuracy[1])
    
    

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Set up logging for predictions
    tensors_to_log = {"accuracy": accuracy[1]}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=100)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, training_hooks=[logging_hook])

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": accuracy}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



def run_graph(X_train_, y_train_, X_test_, y_test_, log_dir, epochs):
    with tf.Graph().as_default() as g:
            # create input functions for train and evaluate methods.
            train_input_fn = tf.estimator.inputs.numpy_input_fn(
                x={"x": X_train_},
                y=y_train_,
                batch_size=40,
                num_epochs=None,
                shuffle=True)
            eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                x={"x": X_test_},
                y=y_test_,
                num_epochs=1,
                shuffle=False)

            # Create an estimator
            classifier = tf.estimator.Estimator(
                model_fn=inceptionv3_model_fn, model_dir=log_dir)
            
            for i in range(epochs):
                print("Epoc %i..." % i)
                # Train network.
                classifier.train(
                    input_fn=train_input_fn,
                    steps=200)

                # Evaluate the model and print results.
                eval_results = classifier.evaluate(input_fn=eval_input_fn)

run_graph(X_train, y_train, X_test, y_test, "cameliyon", 50)
run_graph(X_processed_train, y_train, X_processed_test, y_test, "cameliyon_processed", 50)